In [1]:
import pickle
from nltk.lm import MLE, Laplace, Lidstone, WittenBellInterpolated, StupidBackoff
from classifier import NGramLanguageModel
import sys
import os
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm import MLE, WittenBellInterpolated, Lidstone, StupidBackoff
from nltk.lm.models import Laplace, KneserNeyInterpolated
from collections import Counter
import pickle

In [2]:
# Function to load models
def load_model(author_name):
    model_file = f"trained_{author_name}.pkl"
    with open(model_file, 'rb') as f:
        return pickle.load(f)

In [3]:
# Load models for each author
authors = ["austen", "dickens", "tolstoy", "wilde"]  # Replace with your actual author names
models = {}

for author in authors:
    models[author] = load_model(author)

In [4]:
# Define prompts for sample generation
prompts = [
    "It was an ideal place to",
    "Some years ago, never mind how long",
    "I am surprised that",
    "It was the worst",
    "What a "
]


In [5]:
# Dictionary to store generated samples and perplexities for each author
samples = {author: [] for author in authors}
perplexities = {author: [] for author in authors}

# Function to generate text and calculate perplexity for generated text
def generate_and_calculate_perplexity(models, prompts):
    for author, model in models.items():
        print(f"\nGenerating text and calculating perplexities for {author}:")

        for prompt in prompts:
            # Generate text
            generated_text = model.generate_text(prompt, max_words=20)
            samples[author].append(generated_text)
            
            # Calculate perplexity for the **generated text** instead of the prompt
            perplexity = model.calculate_perplexity([generated_text])
            perplexities[author].append(perplexity)
            
            # Display results
            print(f"Prompt: '{prompt}'")
            print(f"Generated text: {generated_text}")
            print(f"Perplexity for generated text: {perplexity:.2f}\n")

# Generate samples and calculate perplexities for generated text
generate_and_calculate_perplexity(models, prompts)


Generating text and calculating perplexities for austen:
Prompt: 'It was an ideal place to'
Generated text: it was an ideal place to miss bingley ,
Perplexity for generated text: 122.17

Prompt: 'Some years ago, never mind how long'
Generated text: some years ago , never mind how long has this been known all over allenham house , amidst the rough coppice-wood which bordered
Perplexity for generated text: 159.40

Prompt: 'I am surprised that'
Generated text: i am surprised that is , it was true , ” said emma , “ but you would have been spent in joint affliction
Perplexity for generated text: 67.65

Prompt: 'It was the worst'
Generated text: it was the worst and meanest degree of vanity can
Perplexity for generated text: 166.32

Prompt: 'What a '
Generated text: what a contrariety of emotion they excited .
Perplexity for generated text: 155.09


Generating text and calculating perplexities for dickens:
Prompt: 'It was an ideal place to'
Generated text: it was an ideal place to stand up 

In [17]:
from tabulate import tabulate
import textwrap

# Function to truncate text to a specified length
def truncate_text(text, length=50):
    return text if len(text) <= length else text[:length] + '...'

# Function to wrap text to a specified width
def wrap_text(text, width=50):
    return "\n".join(textwrap.wrap(text, width))

# Function to create and display tables for each author's predictions and perplexity scores
def display_perplexity_tables(samples, perplexities, truncate_length=50, wrap_width=50):
    for author in samples.keys():
        # Prepare table data
        table_data = []
        for prompt, generated_text, perplexity in zip(prompts, samples[author], perplexities[author]):
            # Truncate or wrap the generated text
            truncated_text = truncate_text(generated_text, truncate_length)
            wrapped_text = wrap_text(truncated_text, wrap_width)

            table_data.append([wrap_text(prompt, wrap_width), wrapped_text, f"{perplexity:.2f}"])

        # Print the table using tabulate
        print(f"\nTable for {author}:\n")
        print(tabulate(table_data, headers=['Prompt', 'Generated Text', 'Perplexity'], tablefmt='grid'))

# Display the tables for each author with specified truncate and wrap settings
display_perplexity_tables(samples, perplexities, truncate_length=100, wrap_width=50)


Table for austen:

+-------------------------------------+----------------------------------------------------+--------------+
| Prompt                              | Generated Text                                     |   Perplexity |
+=====================================+====================================================+==============+
| It was an ideal place to            | it was an ideal place to miss bingley ,            |       122.17 |
+-------------------------------------+----------------------------------------------------+--------------+
| Some years ago, never mind how long | some years ago , never mind how long has this been |       159.4  |
|                                     | known all over allenham house , amidst the rough   |              |
|                                     | ...                                                |              |
+-------------------------------------+----------------------------------------------------+--------------+
| I am s